# Importing the requried libraries

In [ ]:
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import svm
from sklearn import neighbors
from numpy import random
import warnings
warnings.filterwarnings("ignore")

# SVM Cifar 10

In [ ]:
classesName = ['plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck']

(xTrain, yTrain), (xTest, yTest) = cifar10.load_data()
xVal = xTrain[49000:, :].astype(float)
yVal = np.squeeze(yTrain[49000:, :])
xTrain = xTrain[:49000, :].astype(float)
yTrain = np.squeeze(yTrain[:49000, :])
yTest = np.squeeze(yTest)
xTest = xTest.astype(float)

# Show dimension for each variable
print('Train image shape:    {0}'.format(xTrain.shape))
print('Train label shape:    {0}'.format(yTrain.shape))
print('Test image shape:     {0}'.format(xTest.shape))
print('Test label shape:     {0}\n'.format(yTest.shape))

# Pre processing data
# Normalize the data by subtract the mean image
meanImage = np.mean(xTrain, axis=0)
xTrain -= meanImage
xVal -= meanImage
xTest -= meanImage

# Reshape data from channel to rows
xTrain = np.reshape(xTrain, (xTrain.shape[0], -1))
xVal = np.reshape(xVal, (xVal.shape[0], -1))
xTest = np.reshape(xTest, (xTest.shape[0], -1))

# Add bias dimension columns
xTrain = np.hstack([xTrain, np.ones((xTrain.shape[0], 1))])
xVal = np.hstack([xVal, np.ones((xVal.shape[0], 1))])
xTest = np.hstack([xTest, np.ones((xTest.shape[0], 1))])

class Svm (object):
    """" Svm classifier """

    def __init__(self, inputDim, outputDim):
        self.W = None
        sigma = 0.01
        self.W = sigma * np.random.randn(inputDim, outputDim)

    def calLoss(self, x, y, reg):

        loss = 0.0
        dW = np.zeros_like(self.W)

        s = x.dot(self.W)
        # Score with yi
        s_yi = s[np.arange(x.shape[0]), y]
        # finding the delta
        delta = s - s_yi[:, np.newaxis]+1
        # loss for samples
        loss_i = np.maximum(0, delta)
        loss_i[np.arange(x.shape[0]), y] = 0
        loss = np.sum(loss_i)/x.shape[0]
        # Loss with regularization
        loss += reg*np.sum(self.W*self.W)
        # Calculating ds
        ds = np.zeros_like(delta)
        ds[delta > 0] = 1
        ds[np.arange(x.shape[0]), y] = 0
        ds[np.arange(x.shape[0]), y] = -np.sum(ds, axis=1)

        dW = (1/x.shape[0]) * (x.T).dot(ds)
        dW = dW + (2 * reg * self.W)
        return loss, dW

    def train(self, x, y, lr=1e-4, reg=1e-5, iter=100, batchSize=200, verbose=False):
        # Run stochastic gradient descent to optimize W.
        lossHistory = []
        for i in range(iter):
            xBatch = None
            yBatch = None

            num_train = np.random.choice(x.shape[0], batchSize)
            xBatch = x[num_train]
            yBatch = y[num_train]
            loss, dW = self.calLoss(xBatch, yBatch, reg)
            self.W = self.W - lr * dW
            lossHistory.append(loss)

            if verbose and i % 250 == 0 and len(lossHistory) is not 0:
                print('Loop {0} ; Loss {1}'.format(i, lossHistory[i]))

        pass

    def predict(self, x,):
        yPred = np.zeros(x.shape[0])
        s = x.dot(self.W)
        yPred = np.argmax(s, axis=1)
        return yPred

    def calAccuracy(self, x, y):
        acc = 0
        yPred = self.predict(x)
        acc = np.mean(y == yPred)*100
        return acc

numClasses = np.max(yTrain) + 1

classifier = Svm(xTrain.shape[1], numClasses)

# Training classifier
startTime = time.time()
classifier.train(xTrain, yTrain, lr=1e-7, reg=5e4, iter=1500, verbose=1)

Train image shape:    (49000, 32, 32, 3)
Train label shape:    (49000,)
Test image shape:     (10000, 32, 32, 3)
Test label shape:     (10000,)

Loop 0 ; Loss 155535.5139038931
Loop 250 ; Loss 1022.6662462631962
Loop 500 ; Loss 11.886042221921999
Loop 750 ; Loss 5.703022373802461
Loop 1000 ; Loss 6.186302518223705
Loop 1250 ; Loss 5.50030716347562


In [ ]:
print('Training accuracy:   {0} %'.format(classifier.calAccuracy(xTrain, yTrain)))
print('Validating accuracy: {0} %'.format(classifier.calAccuracy(xVal, yVal)))
print('Testing accuracy:    {0} %'.format(classifier.calAccuracy(xTest, yTest)))

print('Program Execution Time(GPU): {0} seconds'.format(time.time() - startTime))

Training accuracy:   35.68775510204082 %
Validating accuracy: 36.199999999999996 %
Testing accuracy:    35.099999999999994 %
Program Execution Time(GPU): 6.611586570739746 seconds


# SVM Fashion

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

def svm_model():
    model = svm.SVC(C=1, kernel='rbf', gamma=0.02)
    return model

train_size_svm = 20000
test_size_svm = 1000

x_train_svm = x_train[:train_size_svm].reshape(train_size_svm, 28*28)
y_train_svm = y_train[:train_size_svm]
x_test_svm = x_test[:test_size_svm].reshape(test_size_svm, 28*28)
y_test_svm = y_test[:test_size_svm]

startTime = time.time()
svm_model = svm_model()
svm_model.fit(x_train_svm, y_train_svm)

SVC(C=1, gamma=0.02)

In [ ]:
print('Training Accuracy: ' ,(svm_model.score(x_train_svm, y_train_svm))*100, '%')
print('Test Accuracy: ' ,(svm_model.score(x_test_svm, y_test_svm))*100, '%')

print('Program Execution Time(GPU): {0} seconds'.format(time.time() - startTime))

Training Accuracy:  93.24 %
Test Accuracy:  87.7 %
Program Execution Time(GPU): 149.04493975639343 seconds


# SVM Mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

def svm_model():
    model = svm.SVC(C=1, kernel='rbf', gamma=0.02)
    return model

train_size_svm = 20000
test_size_svm = 1000

startTime = time.time()
x_train_svm = x_train[:train_size_svm].reshape(train_size_svm, 28*28)
y_train_svm = y_train[:train_size_svm]
x_test_svm = x_test[:test_size_svm].reshape(test_size_svm, 28*28) 
y_test_svm = y_test[:test_size_svm]

svm_model = svm_model()
svm_model.fit(x_train_svm, y_train_svm)

SVC(C=1, gamma=0.02)

In [ ]:
print('Training Accuracy: ',(svm_model.score(x_train_svm, y_train_svm))*100 , '%')    
print('Test Accuracy: ',(svm_model.score(x_test_svm, y_test_svm))*100, '%')

print('Program Execution Time(GPU): {0} seconds'.format(time.time() - startTime))

Training Accuracy:  99.35000000000001 %
Test Accuracy:  96.89999999999999 %
Program Execution Time(GPU): 105.87819480895996 seconds
